# Test Draft Law scraper

### load libraries & functions

In [1]:
import sys
import os

# Add the parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

In [3]:
from utils import notification_scraper

### get cases from db

In [7]:
from sqlalchemy import create_engine, Column, String, Integer, Table, MetaData
from sqlalchemy.orm import sessionmaker
import pandas as pd

# Define the path to the SQLite database
db_path = "sqlite:///../data/scraper_data.db"

# Create an engine
engine = create_engine(db_path)

# Define a metadata instance
metadata = MetaData()

# Reflect the table structure from the database
tris_scraper_stack = Table("tris_scraper_stack", metadata, autoload_with=engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Query entries where status is "not scraped"
query = session.query(tris_scraper_stack).filter_by(status="not scraped")

# Load query results into a DataFrame
df = pd.DataFrame([dict(row._asdict()) for row in query])

# Close the session
session.close()

# Display the DataFrame (optional)


In [8]:
# Display the DataFrame
### for testing purposes this script only downloads the first:
df = df.head(1)
df

,id,tris_id,status,message_number,email_from,email_date,end_date,url
0,1,2023/0503/SE,not scraped,6670,EU CORPORATE NOTIFICATION SYSTEM\r\n\t<EU-CORP...,2023-08-23,2023-09-27,https://technical-regulation-information-syste...


### execute scraper

In [9]:
folder_l = ['notification/','contributions/',
                'other_messages/'
                ]

tris_selectors = {'draft': "#D-links-container", 'final': "#F-links-container"}
message_selector = "//div[@class='ecl-col-12 ecl-u-border-all ecl-u-border-color-grey-50']"

In [10]:
#df.apply(lambda x: main_tris_page(x['url'], x['tris_id']), axis=1)
import pandas as pd
import asyncio

# Define an async wrapper for DataFrame application
async def apply_main_tris_page(df):
    tasks = [notification_scraper.download_draft(row['url'], row['tris_id']) for _, row in df.iterrows()]
    return await asyncio.gather(*tasks)

# Run the async wrapper
results = await apply_main_tris_page(df)
df['result'] = results

In [11]:
import pandas as pd
import asyncio

# Async wrapper to run contrib_page for each row in the DataFrame
async def process_contrib_pages(df):
    # Use list comprehension to apply contrib_page to each row asynchronously
    results = await asyncio.gather(*(notification_scraper.contrib_page(row['url'], row['tris_id']) for _, row in df.iterrows()))
    df['contributions'] = results
    return df

# Run the async wrapper
df = await process_contrib_pages(df)

count_contribs 0
